## Scenario 1: A single data scientist participating in an ML competition

This scenario demonstrates how an individual data scientist can use MLflow to track machine learning experiments on their local machine. This is a common setup for solo projects, hackathons, or competitions where collaboration and remote access are not required.

### MLflow setup overview
- **Tracking server:** Not used (runs locally, no remote server)
- **Backend store:** Local filesystem (stores experiment metadata in the `mlruns/` folder)
- **Artifacts store:** Local filesystem (stores model files and other artifacts in the same `mlruns/` folder)

With this setup, all experiment runs, parameters, metrics, and artifacts are saved locally. You can explore and compare your experiments using the MLflow UI.

### How to use the MLflow UI
- You can launch the MLflow UI by running `mlflow ui` in your terminal, or by running the provided code cell in this notebook.
- The UI will be available at [http://localhost:5000](http://localhost:5000).
- Use the UI to browse experiments, compare runs, and inspect logged models and artifacts.

> **Tip:** This local setup is ideal for learning and prototyping. For team projects or production, you would typically use a remote tracking server and a more robust backend (e.g., a database and cloud storage).

In [1]:
%pip install mlflow
import mlflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///Users/laurabarros/Desktop/MLOps/nyc-taxis/03-experiment-tracking/mlruns'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///Users/laurabarros/Desktop/MLOps/nyc-taxis/03-experiment-tracking/mlruns/0', creation_time=1761734517946, experiment_id='0', last_update_time=1761734517946, lifecycle_stage='active', name='Default', tags={}>]

### Creating an experiment and logging a new run

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import mlflow
import sklearn
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

mlflow.set_experiment("my-experiment-1")

X, y = load_iris(return_X_y=True)
class_names = load_iris().target_names

models = [
    ("LogisticRegression", LogisticRegression(C=1.0, random_state=42, max_iter=1000, solver="lbfgs")),
    ("RandomForestClassifier", RandomForestClassifier(n_estimators=100, random_state=42))
]

for model_name, model in models:
    with mlflow.start_run() as run:
        mlflow.set_tag("model_type", model_name)
        mlflow.log_param("sklearn_version", sklearn.__version__)
        model.fit(X, y)
        y_pred = model.predict(X)
        acc = accuracy_score(y, y_pred)
        mlflow.log_metric("accuracy", acc)
        # Log confusion matrix as labeled DataFrame
        cm = confusion_matrix(y, y_pred)
        cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
        cm_df.to_csv("confusion_matrix_labeled.csv")
        mlflow.log_artifact("confusion_matrix_labeled.csv")
        # Confusion matrix heatmap as image
        plt.figure(figsize=(5,4))
        sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
        plt.title(f"Confusion Matrix ({model_name})")
        plt.ylabel("True label")
        plt.xlabel("Predicted label")
        plt.tight_layout()
        plt.savefig("confusion_matrix_heatmap.png")
        plt.close()
        mlflow.log_artifact("confusion_matrix_heatmap.png")
        # Provide input_example and use 'name' instead of deprecated 'artifact_path'
        input_example = np.expand_dims(X[0], axis=0)
        mlflow.sklearn.log_model(model, name="models", input_example=input_example)
        mlflow.set_tag("n_classes", len(np.unique(y)))
        mlflow.set_tag("run_time", datetime.datetime.now().isoformat())
        mlflow.set_tag("description", f"{model_name} on Iris dataset")
        print(f"Logged run for {model_name}, accuracy={acc:.3f}")

2025/10/29 11:42:04 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


Logged run for LogisticRegression, accuracy=0.973
Logged run for RandomForestClassifier, accuracy=1.000


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///Users/laurabarros/Desktop/MLOps/nyc-taxis/03-experiment-tracking/mlruns/787603175250119924', creation_time=1761734524979, experiment_id='787603175250119924', last_update_time=1761734524979, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///Users/laurabarros/Desktop/MLOps/nyc-taxis/03-experiment-tracking/mlruns/0', creation_time=1761734517946, experiment_id='0', last_update_time=1761734517946, lifecycle_stage='active', name='Default', tags={}>]

In [ ]:
# Launch MLflow UI (default: uses local mlruns/ folder)
import subprocess
import sys

# This will run 'mlflow ui' as a background process
subprocess.Popen([sys.executable, '-m', 'mlflow', 'ui'])
print("MLflow UI started. Open http://localhost:5000 in your browser.")

MLflow UI started. Open http://localhost:5000 in your browser.


[MLflow] Security middleware enabled with default settings (localhost-only). To allow connections from other hosts, use --host 0.0.0.0 and configure --allowed-hosts and --cors-allowed-origins.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)
INFO:     Started parent process [22785]
INFO:     Started server process [22790]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Started server process [22787]
INFO:     Waiting for application startup.
INFO:     Started server process [22789]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Application startup complete.
INFO:     Started server process [22788]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


### Interacting with the model registry

In [7]:
from mlflow.tracking import MlflowClient


client = MlflowClient()

In [9]:
from mlflow.exceptions import MlflowException

try:
    client.search_registered_models()
except MlflowException:
    print("It's not possible to access the model registry :(")

INFO:     127.0.0.1:49630 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:49630 - "GET /static-files/static/js/main.03f3f89d.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:49631 - "GET /static-files/static/css/main.280d6c90.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:49630 - "GET /static-files/static/js/3617.10568100.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:49631 - "GET /static-files/static/js/5759.45405231.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:49634 - "GET /static-files/static/js/762.6e8d62ec.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:49633 - "GET /static-files/static/css/762.26533251.chunk.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:49631 - "GET /static-files/favicon.ico HTTP/1.1" 200 OK
INFO:     127.0.0.1:49633 - "GET /static-files/manifest.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:49633 - "GET /ajax-api/2.0/mlflow/experiments/search?max_results=5&order_by=last_update_time+DESC HTTP/1.1" 200 OK


INFO:     127.0.0.1:49651 - "GET /ajax-api/2.0/mlflow/experiments/search?max_results=5&order_by=last_update_time+DESC HTTP/1.1" 200 OK
INFO:     127.0.0.1:49671 - "GET /ajax-api/2.0/mlflow/experiments/search?max_results=5&order_by=last_update_time+DESC HTTP/1.1" 200 OK
INFO:     127.0.0.1:49674 - "GET /static-files/static/css/4783.26533251.chunk.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:49675 - "GET /static-files/static/js/4783.ecc05eb2.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:49675 - "GET /ajax-api/2.0/mlflow/experiments/search?max_results=25&order_by=last_update_time+DESC HTTP/1.1" 200 OK
INFO:     127.0.0.1:49675 - "GET /static-files/static/js/9460.396b29fd.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:49674 - "GET /static-files/static/js/8975.6d6dd5b6.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:49679 - "GET /static-files/static/js/683.b5476dee.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:49682 - "GET /static-files/static/js/8911.72a138f6.chunk.js HTTP/1.1" 200 OK
INFO:     1